In [3]:
import numpy as np
import onnx
import onnxruntime as ort
from transformers import AutoTokenizer

# Load the tokenizer and ONNX model once at startup
model_name = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Path to the ONNX model
onnx_model_path = '../model/model.onnx'

# Create an inference session
ort_session = ort.InferenceSession(onnx_model_path)

def extract_layers(model_path):
    """
    Extracts all layers (nodes) from an ONNX model.

    Parameters:
        model_path (str): Path to the ONNX model file.

    Returns:
        List[Dict]: A list of dictionaries containing layer information.
    """
    # Load the ONNX model
    model = onnx.load(model_path)
    layers = []

    # Iterate through all nodes in the graph
    for idx, node in enumerate(model.graph.node):
        layer_info = {
            'layer_index': idx,
            'name': node.name if node.name else f"Unnamed_{idx}",
            'op_type': node.op_type,
            'input': node.input,
            'output': node.output,
            'attributes': {attr.name: onnx.helper.get_attribute_value(attr) for attr in node.attribute}
        }
        layers.append(layer_info)

    return layers

# Example usage
if __name__ == "__main__":
    layers = extract_layers(onnx_model_path)
    print(f"Total Layers: {len(layers)}\n")
    for layer in layers:
        print(f"Layer {layer['layer_index']}:")
        print(f"  Name      : {layer['name']}")
        print(f"  Op Type   : {layer['op_type']}")
        print(f"  Inputs    : {layer['input']}")
        print(f"  Outputs   : {layer['output']}")
        print(f"  Attributes : {layer['attributes']}\n")


/Users/mahdi/repos/LLMviz.io/llmviz/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

/Users/mahdi/repos/LLMviz.io/llmviz/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



Total Layers: 2600

Layer 0:
  Name      : /transformer/Shape
  Op Type   : Shape
  Inputs    : ['input_ids']
  Outputs   : ['/transformer/Shape_output_0']
  Attributes : {}

Layer 1:
  Name      : /transformer/Constant
  Op Type   : Constant
  Inputs    : []
  Outputs   : ['/transformer/Constant_output_0']
  Attributes : {'value': data_type: 7
raw_data: "\001\000\000\000\000\000\000\000"
}

Layer 2:
  Name      : /transformer/Gather
  Op Type   : Gather
  Inputs    : ['/transformer/Shape_output_0', '/transformer/Constant_output_0']
  Outputs   : ['/transformer/Gather_output_0']
  Attributes : {'axis': 0}

Layer 3:
  Name      : /transformer/Constant_1
  Op Type   : Constant
  Inputs    : []
  Outputs   : ['/transformer/Constant_1_output_0']
  Attributes : {'value': dims: 1
data_type: 7
raw_data: "\377\377\377\377\377\377\377\377"
}

Layer 4:
  Name      : Constant_955
  Op Type   : Constant
  Inputs    : []
  Outputs   : ['onnx::Unsqueeze_155']
  Attributes : {'value': dims: 1
data_ty

In [5]:
import numpy as np
import plotly.graph_objects as go

# Example layers with activation data
layers = [
    {
        'layer_index': 0,
        'name': 'Input',
        'op_type': 'Input',
        'inputs': [],
        'outputs': ['input_tensor'],
        'attributes': {},
        'activations': np.random.rand(5, 5)
    },
    {
        'layer_index': 1,
        'name': 'MatMul_1',
        'op_type': 'MatMul',
        'inputs': ['input_tensor', 'weight1'],
        'outputs': ['matmul_output'],
        'attributes': {},
        'activations': np.random.rand(5, 5)
    },
    {
        'layer_index': 2,
        'name': 'Add_1',
        'op_type': 'Add',
        'inputs': ['matmul_output', 'bias1'],
        'outputs': ['add_output'],
        'attributes': {},
        'activations': np.random.rand(5, 5)
    },
    {
        'layer_index': 3,
        'name': 'Relu_1',
        'op_type': 'Relu',
        'inputs': ['add_output'],
        'outputs': ['relu_output'],
        'attributes': {},
        'activations': np.random.rand(5, 5)
    },
    # Add more layers as needed
]




# Initialize figure
fig = go.Figure()

# Function to add operation blocks
def add_operation_block(fig, layer_x, operation_y, op_type, z=0):
    size = 0.4
    fig.add_trace(go.Mesh3d(
        x=[layer_x - size, layer_x + size, layer_x + size, layer_x - size,
           layer_x - size, layer_x + size, layer_x + size, layer_x - size],
        y=[operation_y - size, operation_y - size, operation_y + size, operation_y + size,
           operation_y - size, operation_y - size, operation_y + size, operation_y + size],
        z=[z, z, z, z, z + size, z + size, z + size, z + size],
        color='lightblue',
        opacity=0.6,
        name=op_type,
        hoverinfo='text',
        text=op_type
    ))
    # Add text label
    fig.add_trace(go.Scatter3d(
        x=[layer_x],
        y=[operation_y],
        z=[z + size + 0.1],
        mode='text',
        text=[op_type],
        showlegend=False
    ))

# Function to add activation bars
def add_activation_bars(fig, layer_x, activations, layer_index):
    num_rows, num_cols = activations.shape
    bar_spacing = 0.3
    for i in range(num_rows):
        for j in range(num_cols):
            x = layer_x
            y = i * bar_spacing
            z = j * bar_spacing
            fig.add_trace(go.Scatter3d(
                x=[x, x],
                y=[y, y],
                z=[z, z + activations[i, j]],
                mode='lines',
                line=dict(color='orange', width=4),
                showlegend=False
            ))

# Add layers and their components
for layer in layers:
    layer_x = layer['layer_index']
    op_type = layer['op_type']
    
    # Visualize operation
    add_operation_block(fig, layer_x, 0, op_type, z=0)
    
    # Visualize activations
    add_activation_bars(fig, layer_x, layer['activations'], layer['layer_index'])

# Define layout
fig.update_layout(
    scene=dict(
        xaxis=dict(title='Layers', tickvals=list(range(len(layers))), ticktext=[layer['name'] for layer in layers]),
        yaxis=dict(title='Operations', range=[-1, 2]),
        zaxis=dict(title='Activation Values', range=[0, 1]),
        aspectratio=dict(x=1, y=1, z=0.5),
    ),
    title="3D Visualization of Model's Computational Flow",
    showlegend=False
)

# Show figure
fig.show()

